# Fundamentals of Data Science
Winter Semester 2021

## Prof. Francesco Pinto, Prof. Vito Collica, Prof. Simone Chieppa, Prof. Matteo Basile, Prof. Giovanni Montobbio
<pinto.1871045@studenti.uniroma1.it>, <collica.2011212@studenti.uniroma1.it>, <chieppa.1846140@studenti.uniroma1.it>, <basile.1760927@studenti.uniroma1.it>, <montobbio.1845035@studenti.uniroma1.it>






## Project part 1: Regression

### Code and Theory 


## Notation

- $x^i$ is the $i^{th}$ feature vector
- $y^i$ is the expected outcome for the $i^{th}$ training example
- $m$ is the number of training examples
- $n$ is the number of features
- $\theta$ is the set of parameters learned by the model
- $\hat{y}$ is the estimation of the quality of the sample according to the model

Let's start by setting up our Python environment and importing the required libraries:

In [1]:
import numpy as np # imports a fast numerical programming library
import scipy as sp # imports stats functions, amongst other things
import pandas as pd # lets us handle data as dataframes
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
import timeit # Used to measure execution time
import os # Used to assess the number of CPUs of the system
from multiprocessing import Pool # Used to parallelize the tests
import matplotlib.pyplot as plt

NUM_CORES = max(os.cpu_count()-2, 2)
# Uncomment to force a specific number of cores
#NUM_CORES = 6

## 1.a - Required equations for linear regression

### Prediction

*   $\hat{y}=\theta^Tx$

### Likelihood

We assume a gaussian distribution for the prediction error:

*   $y^{(i)}=\theta^Tx^{(i)}+\varepsilon^{(i)}$

Where $\varepsilon$ considers all the unmodeled effects, such as random noise  

We also assume that $\varepsilon^{(i)}$ are independent and identically distributed

This allows to write the probability distribution of the output given $\theta$ as:

*   $P(y^{(i)}|x^{(i)};\theta)=\frac{1}{\sqrt{2\pi}\sigma}exp(-\frac{(y^{(i)}-\theta^Tx^{(i)})^2}{2\sigma^2})$

The likelihood function is a measure that explains how well the model fits the data, in this case it varies with the learned $\theta$ values and is computed in the following way:
*   $L(\theta)=P(y|x;\theta)=\prod_{i=1}^m{P(y^{(i)}|x^{(i)};\theta)}=\prod_{i=1}^m{\frac{1}{\sqrt{2\pi}\sigma}exp(-\frac{(y^{(i)}-\theta^Tx^{(i)})^2}{2\sigma^2})}$

### log likelihood

*   $l(\theta)=log(L(\theta))=log(\prod_{i=1}^m{\frac{1}{\sqrt{2\pi}\sigma}exp(-\frac{(y^{(i)}-\theta^Tx^{(i)})^2}{2\sigma^2})})=\sum_{i=1}^m{log({\frac{1}{\sqrt{2\pi}\sigma}})exp(-\frac{(y^{(i)}-\theta^Tx^{(i)})^2}{2\sigma^2})}=\sum_{i=1}^m{[log({\frac{1}{\sqrt{2\pi}\sigma}})+(-\frac{(y^{(i)}-\theta^Tx^{(i)})^2}{2\sigma^2})]}=mlog({\frac{1}{\sqrt{2\pi}\sigma}})+\sum_{i=1}^m(-\frac{(y^{(i)}-\theta^Tx^{(i)})^2}{2\sigma^2})$

Our objective is performing MLE (Maximum Likelihood Estimation), maximizing the value of the log likelihood with respect to $\theta$, for that purpose we can ignore $mlog({\frac{1}{\sqrt{2\pi}\sigma}})$, and in $\sum_{i=1}^m(-\frac{(y^{(i)}-\theta^Tx^{(i)})^2}{2\sigma^2})$ we can ignore the constant $\frac{1}{\sigma^2}$ (we keep $\frac{1}{2}$ as it makes the equation of the gradient cleaner):

$argmax_\theta(l(\theta))=argmax_\theta(mlog({\frac{1}{\sqrt{2\pi}\sigma}})+\sum_{i=1}^m(-\frac{(y^{(i)}-\theta^Tx^{(i)})^2}{2\sigma^2}))\approx argmax_\theta(\frac{1}{2}\sum_{i=1}^m(-(y^{(i)}-\theta^Tx^{(i)})^2))=argmin_\theta(\frac{1}{2}\sum_{i=1}^m((y^{(i)}-\theta^Tx^{(i)})^2))$

Which we are gonna minimize through gradient descent.

### Gradient of the log likelihood function

$\frac{\delta}{\delta\theta_j}\frac{1}{2}\sum_{i=1}^m(y^{(i)}-\theta^Tx^{(i)})^2=\sum_{i=1}^{m}(y^{(i)}-\theta^Tx^{(i)})\cdot (-x_j^{(i)})$

### Gradient descent update rule

$\theta_j =\theta_j-\alpha \frac{\delta}{\delta \theta_j}l(\theta)= \theta_j-\alpha \sum_{i=1}^{m}(y^{(i)}-\theta^Tx^{(i)})\cdot (-x_j^{(i)})$

Where $\alpha$ is the learning rate

#### Implementation of linear regression with Gradient descent




In [2]:
def load_dataset(filename):
  #LOAD THE DATASET
  dataset = pd.read_csv(filename,sep =';')
  df = pd.DataFrame(dataset)
  return np.array(df) 

In [3]:
def divide_Xy(database):
  #DIVIDE THE ARRAY IN X AND Y
  y = database[:,-1]
  X = np.delete(database, -1, 1)
  return X,y

In [4]:
def divide_train_test(dataset):
  #DIVIDE TRAIN AND TEST SET
  np.random.seed(0)
  np.random.shuffle(dataset) #shuffled dataset
  train = dataset[0:int(len(dataset)*0.833)]
  test = dataset[int(len(dataset)*0.833):]
  return train, test


In [5]:
def divide_folds(X):
  #KFOLD
  result = []
  kfold = KFold(5, shuffle=True, random_state=0)
  for train,dev in kfold.split(X):
    X_train, X_dev = X[train], X[dev]
    result.append((X_train,X_dev))
  return result

### Define the core functions for the algorithm

In [6]:
def loss_function(theta,features,target):
    '''
    Function to compute the loss of theta according to data x and label y
    
    Input:
    theta: it's the model parameter matrix.
    features: it's the input data matrix. The shape is (N, H)
    target: the label array
    
    Output:
    loss: the loss of theta according to data x and label y
    '''

    # First of all we compute the predictions for the current values of theta 
    predictions = np.dot(features, theta)
    
    # In the steps below we compute the loss function 
    loss = 0

    for i in range(len(features)): 
      loss += 0.5*((target[i] - predictions[i])**2)
    
    # Finally normalize the loss
    loss = loss/len(features)
    return loss


def predictions(features, theta):
    '''
    Function to compute the predictions for the input features
    
    Input:
    theta: it's the model parameter matrix.
    features: it's the input data matrix. The shape is (N, H)
    
    Output:
    preds: the predictions of the input features
    '''


    # Dot product between theta and the input data
    preds = np.dot(features, theta)      
    return preds


def update_theta(theta, target, preds, features, lr):
    '''
    Function to compute the gradient of the loss
    and then return the updated weights

    Input:
    theta: the model parameter matrix.
    target: the label array
    preds: the predictions of the input features
    features: it's the input data matrix. The shape is (N, H)
    lr: the learning rate
    
    Output:
    theta: the updated model parameter matrix.
    '''

    #sum((yi-preds_i)*xij)
    for j in range(len(theta)):
      gradient = 0
      for i in range(len(features)):
        # Gradient equation
        gradient -= (target[i]-preds[i])*features[i][j]
      # Normalize the gradient
      gradient = gradient/len(features)
      # Update the theta using the gradient computed before
      theta[j]=theta[j]-lr*gradient
    return theta

def gradient_descent(theta, features, target, lr, num_steps):
    '''
    Function to execute the gradient descent algorithm

    Input:
    theta: the model parameter matrix.
    target: the label array
    num_steps: the number of iterations 
    features: the input data matrix. The shape is (N, H)
    lr: the learning rate
    
    Output:
    theta: the final model parameter matrix.
    loss_history: the values of the loss function during the process
    '''

    loss_history = np.zeros(num_steps)
    
    for i in range(num_steps):
      # Compute the sigmoid 
      preds = predictions(features, theta)
      # Perform Gradient Ascent
      theta = update_theta(theta, target, preds, features, lr)
      # Save the value of the log_likelihood for every step
      loss_history[i]=loss_function(theta,features,target)
      
    return theta, loss_history



Let's now apply the function gradient_ascent and print the final theta as well as theta_history 

In [7]:
def compute_accuracy(y_pred, y_true, T):
  sum = 0
  for i in range(len(y_pred)):
    # Check if predicted value is within tolerance distance  the ground truth
    if y_pred[i]-T <= y_true[i] <= y_pred[i]+T:       
      sum +=1
  # Normalize the sum to compute the accuracy
  accuracy = sum/len(y_true)
  return accuracy

In [8]:
# For each folds combination train the model and compute the metrics 
def run_folds_mr(folds, n_iter, alpha, tolerances):
  theta_finals = []
  confusion_matrices = []
  loss_histories = []
  ILMIOPIPO = []
  resultsmie = {}
  for T in tolerances:
    # dict(Tolerance: [Accuracies, Precisions])         
    resultsmie[T] = [[], []]
  for fold in folds:
    train = fold[0]
    dev = fold[1]
    train_X, train_y = divide_Xy(train)
    dev_X, dev_y = divide_Xy(dev)

    # Cast output values to int
    train_y = train_y.astype(int)

    # Train the model
    theta0 = np.zeros(train_X.shape[1])
    theta_final, loss_history = gradient_descent(theta0, train_X, train_y, alpha, n_iter) 
    loss_histories.append(loss_history)
    theta_finals.append(theta_final) 

    # This is the same as T=0.5 and is used for the confusion matrix
    # TODO

    y_pred = predictions(dev_X, theta_final)
    y_true = dev_y.astype(int)
    for T in tolerances:
      # Accuracy for the tolerance T in the current fold
      resultsmie[T][0].append(compute_accuracy(y_pred,y_true, T))
      classes = np.unique(y_true)
      for i in classes:
        # Confusion matrix for individual class i in the current fold
        new_y_pred = convert_to_binary(y_pred, i, T)
        new_y_true = convert_to_binary(y_true, i, T)
        cm = confusion_matrix(new_y_true, new_y_pred)
        # Precisions for the tolerance T in the current fold
        if (cm[1][1] + cm[0][1]) == 0:
          precision = 0
        else:
          precision = cm[1][1] / (cm[1][1] + cm[0][1])
        resultsmie[T][1].append(precision)
  averaged_accuracies = {}
  averaged_precisions = {}
  # Average for all folds
  for T in tolerances: 
    averaged_accuracies[T] = sum(resultsmie[T][0]) / len(resultsmie[T][0])
    averaged_precisions[T] = sum(resultsmie[T][1]) / len(resultsmie[T][1])
  loss_mean = np.mean(loss_histories, axis = 0)
  theta_mean = np.mean(theta_finals, axis = 0)
  return (theta_mean, loss_mean, averaged_accuracies, averaged_precisions)

In [9]:
# Turns multivariate classification into binary classification for class i
def convert_to_binary(array, i, T):
  new_array = []
  for elem in array:
    if elem-T <= i <= elem+T:
      new_array.append(1)
    else:
      new_array.append(0)
  return new_array


In [10]:
# Parallelizes tests for a set of parameters
def run_parallelized_test_mr(datasets, lr_values, n_iter_values, tolerances, num_cores):
    processes = []
    for dataset in datasets:
      # load the dataset
      loaded_dataset = load_dataset(dataset)
      # only need the training set here
      train = divide_train_test(loaded_dataset)[0]
      for lr in lr_values:
          for n_iter in n_iter_values:
              # passing the name of the dataset and the dataset itself as a tuple
              processes.append([(dataset, train), lr, n_iter, tolerances])
    pool = Pool(processes=num_cores)
    return pool.map(test_function_mr, processes)

def parallelize_predefined_tests_mr(tests, tolerances, num_cores):
    processes = []
    datasets_dict = {}
    for test in tests:
        if test[0] not in datasets_dict:
            loaded_dataset = load_dataset(test[0])
            train = divide_train_test(loaded_dataset)[0]
            datasets_dict[test[0]]=train
        processes.append([(test[0], datasets_dict[test[0]]), test[1], test[2], tolerances])            
    pool = Pool(processes=num_cores)
    return pool.map(test_function_mr, processes)

# Runs a single test for a (dataset, lr, n_iter, T) tuple
def test_function_mr(arguments):

  dataset, lr, n_iter, tolerances = arguments

  # Log execution time for each test
  time_start = timeit.default_timer()

  print('running on the dataset ' + str(dataset[0]) + ', lr=' + str(lr) + ', n_iter=' + str(n_iter) + '\n')

  folds = divide_folds(dataset[1])

  # Initialize theta0
  theta0 = np.zeros(dataset[1].shape[1])

  # Run gradient descent
  theta_mean, loss_mean, accuracies, precisions = run_folds_mr(folds,  n_iter, lr, tolerances)

  # Compute elapsed time and accuracy
  time_end = timeit.default_timer()
  elapsed_time = time_end-time_start

  print('done! dataset=' + str(dataset[0]) + ', lr=' + str(lr) + ', n_iter=' + str(n_iter) + ', elapsed time=' + str(elapsed_time) + ', loss=' + str(loss_mean[-1]) + ', accuracy(T=1)=' + str(accuracies[1]) + ', precision(T=1)=' + str(precisions[1]) + ', accuracy(T=0)=' + str(accuracies[0]) + ', precision(T=0)=' + str(precisions[0]) + '\n')
  
  # Return the results
  return [(dataset[0], lr, n_iter), elapsed_time, theta_mean, loss_mean, accuracies, precisions]



In [12]:
# Tests various possible numbers of samples, values of the learning rate and numbers of iterations

# Parameters used to run the tests
datasets = ['winequality-red.csv', 'winequality-white.csv']
lr_values = [0.0001, 7.5e-05]
n_iter_values = [100]
tolerances = [x*0.025 for x in range(0,80)]
# Times the duration of the batch
time_start_test = timeit.default_timer()

# Enable predefined tests
enable_predefined_tests = True

# This is for running custom tests
predefined_tests = []
# Server Vito
predefined_tests.append([('winequality-red.csv', 0.00015, 2000), ('winequality-red.csv', 0.00015, 1000), ('winequality-red.csv', 0.0001, 2000), ('winequality-red.csv', 0.0002, 2000), ('winequality-red.csv', 0.0002, 1000), ('winequality-red.csv', 0.00005, 1000), ('winequality-red.csv', 0.00005, 500), ('winequality-red.csv', 0.00005, 2000), ('winequality-red.csv', 7.5e-05, 1000), ('winequality-red.csv', 7.5e-04, 1000), ('winequality-red.csv', 7.5e-04, 2000), ('winequality-white.csv', 7.5e-05, 2000), ('winequality-white.csv', 6e-05, 1000), ('winequality-white.csv', 0.00001, 1000), ('winequality-white.csv', 0.00001, 2000), ('winequality-red.csv', 0.0001, 3000)])
# Mac Francesco
predefined_tests.append([('winequality-white.csv', 7.5e-05, 3000), ('winequality-white.csv', 2.5e-05, 4000), ('winequality-red.csv', 7.5e-05, 2000), ('winequality-white.csv', 5e-05, 3000)])
# Mac Simone
predefined_tests.append([('winequality-red.csv', 0.00005, 4000), ('winequality-white.csv', 5e-05, 2000), ('winequality-white.csv', 6e-05, 2000), ('winequality-white.csv', 0.00001, 3000)])
predefined_tests.append([('winequality-red.csv', 0.00005, 10), ('winequality-red.csv', 0.0005, 10)])

# Which tests to run
test_index = -1

# Starts the tests
if(enable_predefined_tests):
    test_results = parallelize_predefined_tests_mr(predefined_tests[test_index], tolerances, NUM_CORES)
else:
    test_results = run_parallelized_test_mr(datasets, lr_values, n_iter_values, tolerances, NUM_CORES)

time_end_test = timeit.default_timer()
test_elapsed_time = time_end_test-time_start_test
print('test complete, elapsed time: ' + str(test_elapsed_time))

results_file = open('regression_results_' + str(test_index) + '.txt', 'w')
# Only considers the last value of the loss function for the results
output = []
for result in test_results:
  #output.append([result[0], result[1], result[3][-1], result[4][1], result[5][1]])
  output.append([result[0], result[1], result[3][-1], result[4][1], result[5][1], result[4][0], result[5][0]])
  results_file.write(str(result)+'\n')
results_file.close()
print(output)

running on the dataset winequality-red.csv, lr=5e-05, n_iter=10
running on the dataset winequality-red.csv, lr=0.0005, n_iter=10


done! dataset=winequality-red.csv, lr=0.0005, n_iter=10, elapsed time=16.12989930799813, loss=3.121896117986139, accuracy(T=1)=0.19765706400833544, precision(T=1)=0.4405961401879748, accuracy(T=0)=0.0, precision(T=0)=0.0

done! dataset=winequality-red.csv, lr=5e-05, n_iter=10, elapsed time=16.699103061997448, loss=5.19323592546186, accuracy(T=1)=0.15553490467742387, precision(T=1)=0.4749164016372394, accuracy(T=0)=0.0, precision(T=0)=0.0

test complete, elapsed time: 16.848076260997914
[[('winequality-red.csv', 5e-05, 10), 16.699103061997448, 5.19323592546186, 0.15553490467742387, 0.4749164016372394, 0.0, 0.0], [('winequality-red.csv', 0.0005, 10), 16.12989930799813, 3.121896117986139, 0.19765706400833544, 0.4405961401879748, 0.0, 0.0]]


## Make predictions over the test set

In [ ]:
def run_final_test(test_X, test_y, theta_final, tolerances):
  theta_finals = []
  confusion_matrices = []
  loss_histories = []
  y_pred = predictions(test_X, theta_final)
  y_true = test_y.astype(int)
  metrics = {}
  for T in tolerances:       
    metrics[T] = [0, []]
    # Accuracy for the tolerance T
    metrics[T][0] = compute_accuracy(y_pred,y_true, T)
    classes = np.unique(y_true)
    for i in classes:
      # Confusion matrix for individual class i 
      new_y_pred = convert_to_binary(y_pred, i, T)
      new_y_true = convert_to_binary(y_true, i, T)
      cm = confusion_matrix(new_y_true, new_y_pred)
      # Precisions for the tolerance T
      if (cm[1][1] + cm[0][1]) == 0:
        precision = 0
      else:
        precision = cm[1][1] / (cm[1][1] + cm[0][1])
      metrics[T][1].append(precision)
    metrics[T][1] = sum(metrics[T][1]) / len(metrics[T][1])
  
  confusion_final = confusion_matrix(y_true,np.round_(y_pred))
  return metrics, confusion_final
  

def retrieve_theta(results, dataset, lr, num_iter):
  data = None
  for result in results:
    if result[0][0] == dataset and result[0][1] == lr and result[0][2] == num_iter:
      data = result
      break
  if data == None:
    return "result not found"
  return data[2]




In [ ]:
#TAKE THE TEST SETS
white_set = load_dataset("winequality-white.csv")
red_set = load_dataset("winequality-red.csv")

white_train, white_test = divide_train_test(white_set)
white_test_X,white_test_y = divide_Xy(white_test)

red_train, red_test = divide_train_test(red_set)
red_test_X,red_test_y = divide_Xy(red_test)

theta_final_red = retrieve_theta(test_results, "winequality-red.csv",0.0001,100)
theta_final_white = retrieve_theta(test_results, "winequality-white.csv", 7.5e-05, 100)

metrics_red, cm_red = run_final_test(red_test_X, red_test_y, theta_final_red, tolerances)
metrics_white, cm_white = run_final_test(white_test_X, white_test_y, theta_final_white, tolerances)

cm_red, cm_white


In [ ]:
# Plot the rec curve for specific parameters
# Tests for the given parameters have to be run first
def rec_curve(metrics,name):
  fig,ax = plt.subplots(num=2)
  ax.set_ylabel('Accuracy ' + name)
  ax.set_xlabel('T')
  _=ax.plot([x for x in metrics.keys()],[y[0] for y in metrics.values()],'b.')

rec_curve(metrics_red, "Red")
#rec_curve(metrics_white, "White")


## Project part 2: Classification 

Defining one-hot vectors for the ground truths

In [ ]:
import scipy
import scipy.sparse
import numpy as np

def class2OneHot(vec):
    out_sparse = scipy.sparse.csr_matrix((np.ones(vec.shape[0]), (vec, np.array(range(vec.shape[0])))))
    out_onehot = np.array(out_sparse.todense()).T
    return out_onehot


ALGORITHMS

In [ ]:
def softmax(theta, X):
    softmax = np.zeros((len(X),len(theta[0])))
    for i in range(len(X)):
      sum = 0
      for k in range(len(theta[0])):
        # Compute the denominator of the softmax function following the formula written above
        sum += np.exp(np.dot(X[i],theta[:,k]))
      for j in range(len(theta[0])):
        # Compute the numerator of the softmax function following the formula written above
        num = np.exp(np.dot(X[i], theta[:,j]))
        # Compute the probability dividing the numerator by the denominator
        softmax[i,j] = num/sum
    return softmax


def CELoss(theta, X, y_onehot):

    loss = 0
    softm = softmax(theta,X)
    for i in range(len(y_onehot)):
      for k in range(len(y_onehot[0])):
        if y_onehot[i][k] == 1:
          # Compute the loss where the value of the one hot vector is 1
          loss -= np.log(softm[i][k])
    return loss


def CELoss_jacobian(theta, X, y_onehot):
    
    # In the steps below we compute the gradient of the cross entropy for every
    # Theta and then update the jacobian matrix instead of computing it for one theta at the time
    preds = softmax(theta,X)
    jacobian = np.zeros((len(theta),len(theta[0])))
    for i in range(len(theta[0])):
      # For every row in theta initialize the sum as array of zeros
      sum = np.zeros(len(X[0]))
      for l in range(len(X)):
        # Compute the gradient of the cross entropy loss
        sum -= X[l]*(y_onehot[l][i] - preds[l][i])
      # Normalize the sum
      sum = sum/len(X)
      for j in range(len(jacobian)):
        # Update the value in the jacobian matrix
        jacobian[j][i] = sum[j]
    return jacobian


def gradient_descent(theta, X, y_onehot, alpha=0.01, iterations=100):
    
    # We initialize an empty array to be filled with loss value after each iteration
    loss_history = np.zeros(iterations)
    
    # With a for loop we compute the steps of GD algo
    for it in range(iterations):
        # Compute the softmax
        preds = softmax(theta, X)
        # Compute the jacobian
        Jacob = CELoss_jacobian(theta, X, y_onehot)
        for j in range(len(theta)):
          # Update the value of theta
          theta[j]=theta[j]-alpha*Jacob[j]
        # Save the value of the cross entropy loss for every iteration
        loss_history[it]=CELoss(theta, X, y_onehot)
    return theta, loss_history

Confusion matrix


TP: The actual value and predicted value should be the same. So concerning Setosa class, the value of cell 1 is the TP value.

FN: The sum of values of corresponding rows except the TP value

FP : The sum of values of corresponding column except the TP value.

TN: The sum of values of all columns and row except the values of that class that we are calculating the values for.

In [ ]:
# For each folds combination train the model and compute the metrics
def run_folds_class(folds, n_iter, alpha, T = 0):

  theta_finals = []
  confusion_matrices = []
  loss_histories = []
  for fold in folds:
    train = fold[0]
    dev = fold[1]
    train_X, train_y = divide_Xy(train)
    dev_X, dev_y = divide_Xy(dev)
    train_y = train_y.astype(int)
    train_y_onehot = class2OneHot(train_y)
    
    np.random.seed(0)
    theta0 = np.random.rand(train_X.shape[1], len(train_y_onehot[0]))
    
    theta_final, loss_history = gradient_descent(theta0, train_X, train_y_onehot, alpha, iterations=n_iter)    #era alpha=0.00001
    predictions = softmax(theta_final, dev_X)
    y_pred = np.argmax(predictions, axis=-1)
    dev_y = dev_y.astype(int)
    dev_y_onehot = class2OneHot(dev_y)
    y_true = np.argmax(dev_y_onehot, axis=-1)
    
    cm = confusion_matrix(y_true, y_pred, labels=range(11))
    loss_histories.append(loss_history)
    theta_finals.append(theta_final)
    confusion_matrices.append(cm)

  confusion_mean = np.mean(confusion_matrices, axis = 0)
  loss_mean = np.mean(loss_histories, axis = 0)
  theta_mean = np.mean(theta_finals, axis = 0)
  accuracies = []
  precisions = []
  fp = 0
  T = round(T)
  for i in range(11):
    tp = 0
    for t in range(1, T+1):
      if (i-t >= 0):
        tp += confusion_mean[i,i-t] 
      if (i+t <= len(confusion_mean[0])-1):
        tp += confusion_mean[i,i+t]   
    tp += confusion_mean[i,i]
    if(T==0):
      tp = confusion_mean[i,i]
    fp = sum(confusion_mean[:,i]) - confusion_mean[i,i]
    fn = sum(confusion_mean[i]) - tp
    tn = sum(sum(confusion_mean)) - tp - fp - fn
    accuracies.append((tn + tp)/(tn+tp+fp+fn))
    if (tp + fp) == 0:
      precisions.append(1)
    else:
      precisions.append(tp/(tp + fp))
  accuracies2 = remove_upperclass(accuracies)
  precisions2 = remove_upperclass(precisions)
  return (sum(accuracies2)/len(accuracies2), sum(precisions2)/len(precisions2)), theta_mean
    

In [ ]:
def remove_upperclass(array):
  new_array = []
  for i in range(len(array)):
    if i not in [0,1,10]:
      new_array.append(array[i])
  return new_array


In [ ]:
#def run_parallelized_test(datasets, lr_values, n_iter_values, random_seeds, num_cores):
def run_parallelized_test(datasets, lr_values, n_iter_values,tolerances, num_cores):
    processes = []
    for dataset in datasets:
      for lr in lr_values:
          for n_iter in n_iter_values:
            for T in tolerances:
              #for seed in random_seeds:
                  # Queue a process for each (n_samples, alpha, n_iter, seed) tuple
              #processes.append([dataset, lr, n_iter, seed])
              processes.append([dataset, lr, n_iter, T])
    pool = Pool(processes=num_cores)
    return pool.map(test_function, processes)

def parallelize_predefined_tests(processes, num_cores):
    pool = Pool(processes=num_cores)
    return pool.map(test_function, processes)

# Runs a single test for a (dataset, lr, n_iter) tuple
def test_function(arguments):
  dataset, lr, n_iter, T = arguments            
  

  # Log execution time for each test
  time_start = timeit.default_timer()

  print('running on the dataset ' + str(dataset) + ', lr=' + str(lr) + ', n_iter=' + str(n_iter) + '\n')

  # Generate the data based on the seed
  data = load_dataset(dataset)                 

  train, test = divide_train_test(data)

  test_X,test_y = divide_Xy(test)

  folds = divide_folds(train)
  

  # Generate random initial theta values based on the seed
  #np.random.seed(seed)
  #theta0 = np.random.rand(X.shape[1], len(np.unique(y)))
  # Initialize theta0
  #theta0 = np.zeros(X.shape[1])

  # Run gradient descent
  #theta_final, loss_history = gradient_descent(theta0, X, y, lr, n_iter)
  metrics, theta_final = np.array(run_folds_class(folds,  n_iter, lr, T))                

  # Compute elapsed time and accuracy
  time_end = timeit.default_timer()
  elapsed_time = time_end-time_start
  #accuracy = compute_accuracy(theta_final, X, y)

  #print('done! dataset=' + str(dataset) + ', lr=' + str(lr) + ', n_iter=' + str(n_iter) + ', seed=' + str(seed) + ', elapsed time=' + str(elapsed_time) + ', accuracy=' + str(accuracy) + '\n')
  print('done! dataset=' + str(dataset) + ', lr=' + str(lr) + ', n_iter=' + str(n_iter) + ', elapsed time=' + str(elapsed_time) + ', accuracy=' + str(metrics[0]) + ', precision=' + str(metrics[1]) +'\n' )
  
  # Return the results
  #return [(lr, n_iter, seed), elapsed_time, accuracy, log_l_history]
  return [(dataset, lr, n_iter, T), elapsed_time, metrics[0], metrics[1], theta_final]             



In [ ]:
# Tests various possible numbers of samples, values of the learning rate and numbers of iterations

# Initialize theta0
#theta0 = np.zeros(white_x.shape[1])

# Parameters used to run the tests
datasets = ['winequality-white.csv', 'winequality-red.csv']        
lr_values = [ 0.00001, 0.00002, 0.00005, 0.0001]
n_iter_values = [500, 1000, 2000, 3000]
tolerances = [0, 1]

#datasets = [ 'winequality-red.csv']        
#lr_values = [ 0.0001]
#n_iter_values = [10]
#tolerances = [0]

sets = []
# Server, 16 threads Francesco numero curr_set_index = 0
sets.append([['winequality-red.csv', 2e-05, 3000, 1], ['winequality-red.csv', 5e-05, 3000, 1], ['winequality-red.csv', 0.0001, 3000, 1], ['winequality-white.csv', 2e-05, 3000, 0], ['winequality-white.csv', 5e-05, 3000, 0], ['winequality-white.csv', 0.0001, 3000, 0]])
# Desktop, 12 threads Simone numero curr_set_index = 1
sets.append([['winequality-red.csv', 0.0001, 4000, 1], ['winequality-red.csv', 0.0001, 4000, 1], ['winequality-red.csv', 0.0001, 3000, 0], ['winequality-white.csv', 1e-05, 3000, 0], ['winequality-red.csv', 2e-05, 3000, 0], ['winequality-red.csv', 0.0002, 2500, 1], ['winequality-red.csv', 0.0002, 2500, 1]])
# Laptop, 8 threads Vito numero curr_set_index = 2
sets.append([['winequality-white.csv', 1e-05, 500, 1], ['winequality-white.csv', 5e-06, 500, 1], ['winequality-white.csv', 75e-07, 500, 1], ['winequality-white.csv', 1e-05, 1000, 1], ['winequality-white.csv', 1e-05, 2000, 1], ['winequality-white.csv', 2e-05, 1000, 1], ['winequality-white.csv', 5e-06, 1000, 1], ['winequality-white.csv', 75e-07, 1000, 1], ['winequality-red.csv', 0.0001, 2000, 1], ['winequality-red.csv', 0.0002, 2000, 1], ['winequality-red.csv', 0.0005, 1500, 1], ['winequality-white.csv', 5e-05, 2000, 0], ['winequality-white.csv', 5e-06, 1000, 1], ['winequality-white.csv', 75e-07, 1000, 1], ['winequality-white.csv', 0.0003, 2000, 0], ['winequality-red.csv', 2e-05, 2000, 0], ['winequality-red.csv', 5e-05, 2000, 0], ['winequality-red.csv', 0.0002, 2000, 1], ['winequality-red.csv', 0.0005, 1500, 1]])
sets.append([['winequality-white.csv', 0.001, 10, 0], ['winequality-red.csv', 0.001, 10, 0]])
sets.append([['winequality-white.csv', 1e-05, 2000, 0], ['winequality-white.csv', 1e-05, 2000, 1]])
sets.append([['winequality-red.csv', 0.0001, 10, 0], ['winequality-red.csv', 0.0001, 10, 1]])

curr_set_index = -1

# Times the duration of the batch
time_start_test = timeit.default_timer()

# Starts the tests
#results = run_parallelized_test(datasets, lr_values, n_iter_values, tolerances, NUM_CORES)
results = parallelize_predefined_tests(sets[curr_set_index], NUM_CORES)

time_end_test = timeit.default_timer()
test_elapsed_time = time_end_test-time_start_test
print('test complete, elapsed time: ' + str(test_elapsed_time))

print(results)

comment='''# Only considers the last value of the loss function for the results
output = []
#for result in averaged_results:
for result in results:
  output.append([result[0], result[1], result[2], result[3]])
print(output)'''

results_file = open('classification_results_' + str(curr_set_index) + '.txt', 'w')
for result in results:
  results_file.write(str(result)+'\n')
results_file.close()
# Run Gradient Ascent method
#n_iter=1000
#white_theta_final, white_loss_history = gradient_descent(theta0,white_x,white_y,lr=0.0000005,num_steps=n_iter)
#red_theta_final, red_loss_history = gradient_descent(theta0,red_x,red_y,lr=0.0000005,num_steps=n_iter)
#print(white_theta_final)
#print(red_theta_final)
#print(loss_history)

In [ ]:
def to_regression(theta_final, X, y):
  #print(theta_final)
  preds = softmax(theta_final, X)
  y_regr = []
  for predicted_probabilities in preds:
    somma = 0
    divisore = sum(predicted_probabilities)
    for i in range(len(preds[1])):
      somma += i*predicted_probabilities[i]
    y_regr.append(somma/divisore)
  return y_regr

y_regr = to_regression(results[-1][-1], red_test_X, red_test_y)
y_regr

In [ ]:
theta_final = results[-1][-1]
y_pred = to_regression(results[-1][-1], red_test_X, red_test_y)
y_true = red_test_y
resultsmie = {}
for T in tolerances:
  resultsmie[T] = [[], []]
for T in tolerances:
  LAMIASOL = compute_accuracy(theta_final, dev_X, dev_y, T)
  resultsmie[T][0].append(LAMIASOL)
  classes = np.unique(y_true)
  for i in classes:
    new_y_pred = convertitor(y_pred, i, T)
    new_y_true = convertitor(y_true, i, T)
    cm = confusion_matrix(new_y_true, new_y_pred)
    if (cm[1][1] + cm[0][1]) == 0:
      precision = 0
    else:
      precision = cm[1][1] / (cm[1][1] + cm[0][1])
    resultsmie[T][1].append(precision)
averaged_accuracies = {}
averaged_precisions = {}
for T in tolerances:
  averaged_accuracies[T] = sum(resultsmie[T][0]) / len(resultsmie[T][0])
  averaged_precisions[T] = sum(resultsmie[T][1]) / len(resultsmie[T][1])

In [ ]:
def convertitor(array, i, T):
  new_array = []
  for elem in array:
    if elem-T <= i <= elem+T:
      new_array.append(1)
    else:
      new_array.append(0)
  return new_array


REC CURVE

In [ ]:
#DA LEVARE NEL MOMENTO IN CUI UNIAMO I DUE DOCUMENTI

fig,ax = plt.subplots(num=2)
ax.set_ylabel('Accuracy')
ax.set_xlabel('T')
_=ax.plot([x for x in averaged_accuracies.keys()],[y for y in averaged_accuracies.values()],'b.')


